In [0]:
!pip3 install -U tensorflow --quiet

In [0]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import Normalizer
import pandas as pd


In [8]:
from google.colab import files
uploaded = files.upload()

Saving Churn.csv to Churn.csv


In [0]:
dataset = pd.read_csv('Churn.csv')

In [10]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [12]:
#include relevant columns within x and y
x = dataset.iloc[:, 3:13] # let go of customerid, rownumber and surname as they are not useful for prediction
y = dataset.iloc[:, 13]
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [13]:
#deal with categorical data --> encode them

from sklearn.preprocessing import LabelEncoder
labelencoder_x = LabelEncoder()
x.iloc[:, 1] = labelencoder_x.fit_transform(x.iloc[:, 1]) #applying on Geography
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,Female,42,2,0.00,1,1,1,101348.88
1,608,2,Female,41,1,83807.86,1,0,1,112542.58
2,502,0,Female,42,8,159660.80,3,1,0,113931.57
3,699,0,Female,39,1,0.00,2,0,0,93826.63
4,850,2,Female,43,2,125510.82,1,1,1,79084.10


In [14]:
#apply encoder on Gender as well
labelencoder_x_2 = LabelEncoder()
x.iloc[:, 2] = labelencoder_x_2.fit_transform(x.iloc[:, 2]) #applying on Gender
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [0]:
#train and test set split, and feature scaling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
X_train.shape

(8000, 10)

In [0]:
#Initialize model
tf.keras.backend.clear_session()

In [0]:

# build a model using tensorflow keras
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(10,)))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 1 Output after applying softmax
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# as output is binary outcome, and activation should be 'sigmoid'

#If dependent variables have more than two categories, use activation = 'softmax'
#Compile the model - add Loss and Gradient Descent optimizer

#opt_ = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
#print(opt_)

model.compile(optimizer= 'Adam', loss="binary_crossentropy", metrics=['accuracy'])

In [96]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 11        
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000      
_________________________________________________________________
batch_normalization (BatchNo (None, 500)               2000      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
_________________________________________________________________
batch_normalization_2 (Batch (None, 10)                4

In [111]:
model.fit(X_train, Y_train, 
          validation_data=(X_test, Y_test), 
          epochs=100,# 20
          batch_size= 20 ) # 50
#X_train, X_test, Y_train, Y_test

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 186us/sample - loss: 0.3985 - acc: 0.8253 - val_loss: 0.3925 - val_acc: 0.8410
Epoch 2/100
8000/8000 [==============================] - 2s 189us/sample - loss: 0.3993 - acc: 0.8230 - val_loss: 0.3992 - val_acc: 0.8205
Epoch 3/100
8000/8000 [==============================] - 2s 193us/sample - loss: 0.3996 - acc: 0.8245 - val_loss: 0.3876 - val_acc: 0.8185
Epoch 4/100
8000/8000 [==============================] - 2s 209us/sample - loss: 0.3999 - acc: 0.8226 - val_loss: 0.3821 - val_acc: 0.8295
Epoch 5/100
8000/8000 [==============================] - 2s 203us/sample - loss: 0.4012 - acc: 0.8245 - val_loss: 0.3905 - val_acc: 0.8195
Epoch 6/100
8000/8000 [==============================] - 2s 211us/sample - loss: 0.4014 - acc: 0.8225 - val_loss: 0.3822 - val_acc: 0.8255
Epoch 7/100
8000/8000 [==============================] - 2s 211us/sample - loss: 0.3969 - acc: 0.8264 - val_loss: 0.38

In [112]:
test_predictions = model.predict(X_test)
test_predictions.shape

(2000, 1)

In [113]:
test_predictions

array([[0.16655135],
       [0.32826945],
       [0.31130448],
       ...,
       [0.03397021],
       [0.13233325],
       [0.12256646]], dtype=float32)

In [114]:
# predict results using 0.5 as a threshold
y_pred = (test_predictions > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [117]:
print(Y_test)

9394    0
898     1
2398    0
5906    0
2343    0
       ..
1037    0
2899    0
9549    0
2740    0
6690    0
Name: Exited, Length: 2000, dtype: int64


In [115]:
# Compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print(cm)

[[1520   75]
 [ 245  160]]


In [116]:
print (((cm[0][0]+cm[1][1])*100)/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]), '% of testing data was classified correctly')

84.0 % of testing data was classified correctly
